In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
year_month = '202405'

In [3]:
year_month_dt = datetime.strptime(year_month, '%Y%m')

# 计算去年同期的年和月
previous_year = year_month_dt.year - 1
previous_month = year_month_dt.month

Mon = previous_month

# 格式化去年同期的年月
previous_year_month = f"{previous_year:04d}{previous_month:02d}"

# 计算去年同期月份的第一天
previous_month_first_day = year_month_dt.replace(year=previous_year, month=previous_month, day=1)

# 计算去年同期月份的最后一天
etd = (previous_month_first_day + timedelta(days=31)).replace(day=1) - timedelta(days=1)
etd = etd.strftime('%Y-%m-%d')

# 输出结果
print(f"Previous year month: {previous_year_month}")
print(f"ETD (End of the month of the previous year): {etd}")
print(Mon)


Previous year month: 202305
ETD (End of the month of the previous year): 2023-05-31
5


In [4]:
# df_complaint_21_22 =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202401.xlsx',sheet_name=1)
# df_complaint_21_22 = df_complaint_21_22.loc[df_complaint_21_22['Year'].isin([2021,2022])]

In [5]:
df_complaints_2024 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\2024\{} Complaint Data.xlsx'.format(year_month),sheet_name='2024 Complaint Database')
df_complaints_2024.drop(columns='New manufacturing complaints',inplace=True)

In [6]:
df_complaints_2023 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\unidentified\2023\{} Complaint Data.xlsx'.format(previous_year_month),sheet_name='2023 Complaint Database')

In [7]:
df_complaints = pd.concat([df_complaints_2024,df_complaints_2023],ignore_index=True)

In [8]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v2.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)

In [9]:
df_complaints['Notification Created Date'] = pd.to_datetime(df_complaints['Notification Created Date'],format='%Y-%m-%d')

In [10]:
(
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023,2024])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter)
    .pivot_table(
        index = ['Division'],
        columns = ['Year','Quarter'],
        values = ['Notification Number'],
        aggfunc ='count',
        fill_value=0,
        margins=True
    )
)

Notification Number                      
Year                    2023       2024        All
Quarter                    1    2     1    2      
Division                                          
10.0                      75   34   141   47   297
14.0                       0    0    11    1    12
15.0                      70   41    76   36   223
18.0                      41    3    48    4    96
20.0                      11    2    39    3    55
21.0                      10   11    17    8    46
22.0                      68   31   116   42   257
29.0                       5    2    33   20    60
30.0                    1143  181   823  293  2440
32.0                      53   11    72   17   153
33.0                       2    1    12    2    17
34.0                      45    9    99   26   179
35.0                       6   50    93   40   189
40.0                      34   10    42   14   100
41.0                       5    3     9    2    19
49.0                       0    0    12    3    15
50.0                      25    1    39   10    75
51.0                      23    5    11    5    44
55.0                      23    7    11    0    41
60.0                       3    0     7    2    12
65.0                       0    0     5    0     5
71.0                       1    0     2    0     3
72.0                       9    1     1    1    12
75.0                       6    1    25    9    41
81.0                      54   27    43    9   133
82.0                       9    2     4    2    17
All                     1721  433  1791  596  4541

In [11]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023,2024])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(int).map(str),
            )
    .groupby(['Division','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints'},inplace=True)

In [12]:
# df_BUOM_2021  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2021.xlsx', usecols='D,E,AB,AP,BI')
# df_BUOM_2022  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2022.xlsx', usecols='D,E,AB,AP,BI')

In [14]:
# df_BUOM_2023  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2023\US BUOM {}.xlsx'.format(previous_year_month), usecols='C,D,J,R,S')
df_BUOM_2023  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2023\US BUOM {}.xlsx'.format(previous_year_month), usecols='C,D,H,P,Q')
df_BUOM_2023 = df_BUOM_2023.loc[(df_BUOM_2023['ShipmentDt'] <= etd) & (df_BUOM_2023['ShipmentDt'] >= '2023-01-01')]


In [15]:
df_BUOM_2024  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\US\US BUOM {}.xlsx'.format(year_month), usecols='D,E,P,X,Y')

In [16]:
df_BUOM_all = pd.concat([df_BUOM_2024,df_BUOM_2023],ignore_index=True)

In [17]:
df_BUOM_all.dropna(subset=['StrippedVendorNumber'],inplace=True)

In [18]:
df_BUOM_all['Year'] = df_BUOM_all['ShipmentDt'].dt.year
df_BUOM_all['Quarter'] = df_BUOM_all['ShipmentDt'].dt.quarter
df_BUOM_all['Month'] = df_BUOM_all['ShipmentDt'].dt.month

In [19]:
df_BUOM_all.rename(columns={'StrippedVendorNumber':'VendorNumber','ProductDivisionCode':'Division','BUOMShipmentQty':'BUOM'},inplace=True)
df_BUOM_all['Division'] = df_BUOM_all['Division'].map(int).map(str)
df_BUOM_all['VendorNumber'] = df_BUOM_all['VendorNumber'].map(int).map(str)

In [20]:
df_BUOM_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260360 entries, 0 to 260359
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   VendorNumber  260360 non-null  object        
 1   VendorName    260360 non-null  object        
 2   Division      260360 non-null  object        
 3   BUOM          260360 non-null  int64         
 4   ShipmentDt    260360 non-null  datetime64[ns]
 5   Year          260360 non-null  int64         
 6   Quarter       260360 non-null  int64         
 7   Month         260360 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 15.9+ MB


In [21]:
df_BUOM_all

,VendorNumber,VendorName,Division,BUOM,ShipmentDt,Year,Quarter,Month
0,116175,DOUBLE DOVE GROUP CO LTD,14,57000,2024-03-16,2024,1,3
1,105110,SINO PROTECTION HOLDINGS CO LTD,21,28700,2024-03-19,2024,1,3
2,105110,SINO PROTECTION HOLDINGS CO LTD,21,6030,2024-03-19,2024,1,3
3,105110,SINO PROTECTION HOLDINGS CO LTD,21,2250,2024-03-19,2024,1,3
4,105110,SINO PROTECTION HOLDINGS CO LTD,21,1692,2024-03-19,2024,1,3
...,...,...,...,...,...,...,...,...
260355,104858,EMERALD NONWOVENS INTL CO LTD,51,15000,2023-02-03,2023,1,2
260356,104858,EMERALD NONWOVENS INTL CO LTD,15,8064,2023-02-03,2023,1,2
260357,104858,EMERALD NONWOVENS INTL CO LTD,51,3700,2023-02-03,2023,1,2
260358,104858,EMERALD NONWOVENS INTL CO LTD,51,9450,2023-02-03,2023,1,2


In [22]:
vendor_mapping_inspection = vendor_mapping.loc[vendor_mapping['Is_Asia']=='Y','Vendor Number'].to_list()
df_BUOM_all = df_BUOM_all.loc[df_BUOM_all['VendorNumber'].isin(vendor_mapping_inspection)]

In [23]:
# df_BUOM_2022.loc[(df_BUOM_2022['VendorName'] == 'ALPHA PLASTIC TECH CO LTD') & (df_BUOM_2022['ProductDivisionCode'] == 10) & (df_BUOM_2022['ShipmentDt'] <= '2022-01-31')]

In [24]:
df_BUOM_gy = df_BUOM_all.groupby(['Division','Year','Quarter','Month'],as_index=False)['BUOM'].sum()

In [25]:
df_BUOM_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Division  260 non-null    object
 1   Year      260 non-null    int64 
 2   Quarter   260 non-null    int64 
 3   Month     260 non-null    int64 
 4   BUOM      260 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 10.3+ KB


In [26]:
df_complaints_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Division    207 non-null    object
 1   Year        207 non-null    int64 
 2   Quarter     207 non-null    int64 
 3   Month       207 non-null    int64 
 4   Complaints  207 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 8.2+ KB


In [27]:
df_cpm = pd.merge(df_complaints_gy,df_BUOM_gy,on=['Division','Year','Quarter','Month'],how='outer').fillna(0)

In [28]:
df_cpm.to_excel(f'./out_{year_month}.xlsx')

In [140]:
pivot_table  = (
    df_cpm
    .pivot_table(
        index = ['Division'],
        columns = ['Year','Quarter'],
        values = ['Complaints', 'BUOM'], # 指定想要运算的列
        aggfunc = {'Complaints': 'sum', 'BUOM': 'sum'},
        fill_value = 0
    )
)


In [141]:
pivot_table.to_excel('pivot_table.xlsx')